Credit goes to https://colab.research.google.com/github/count0/colab-gt/blob/master/colab-gt.ipynb for the steps to use graph_tool in Colab.

In [1]:
!echo "deb http://downloads.skewed.de/apt jammy main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard
!pip install gravis

Executing: /tmp/apt-key-gpghome.H2aw9lbtSc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
gpg: key 612DEFB798507F25: public key "Tiago de Paula Peixoto <tiago@skewed.de>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:7 http://downloads.skewed.de/apt jammy InRelease [7,535 B]
Hit:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd6

In [2]:
from graph_tool.all import *

class MyList:
    def __init__(self, arr):
        self.arr = arr
    # https://stackoverflow.com/questions/1047021/overriding-in-python-iadd-method
    # https://stackoverflow.com/questions/5082190/typeerror-after-overriding-the-add-method
    def __add__(self,other_arr):
        if type(other_arr) == list:
            self.arr.extend(other_arr)
        if type(other_arr) == MyList:
            self.arr.extend(other_arr.arr)
        return self
    def __radd__(self,other_arr):
        if type(other_arr) == list:
            self.arr.extend(other_arr)
        if type(other_arr) == MyList:
            self.arr.extend(other_arr.arr)
        return self

# g = Graph(directed=False)
# for i in range(16):
#     g.add_vertex()
# g.add_edge(0, 2)
# g.add_edge(0, 3)
# g.add_edge(0, 4)
# g.add_edge(0, 5)
# g.add_edge(1, 2)
# g.add_edge(1, 4)
# g.add_edge(1, 7)
# g.add_edge(2, 4)
# g.add_edge(2, 6)
# g.add_edge(3, 7)
# g.add_edge(4, 10)
# g.add_edge(5, 7)
# g.add_edge(5, 11)
# g.add_edge(6, 7)
# g.add_edge(6, 11)
# g.add_edge(8, 9)
# g.add_edge(8, 10)
# g.add_edge(8, 11)
# g.add_edge(8, 14)
# g.add_edge(8, 15)
# g.add_edge(9, 12)
# g.add_edge(10, 11)
# g.add_edge(10, 12)
# g.add_edge(11, 13)
# g.add_edge(13, 14)

# vprop = g.new_vertex_property("object")
# vprop[v1] = MyList([i for i in range(4)])
# vprop[v2] = MyList([i for i in range(100, 107)])
# vprop[v3] = MyList([i for i in range(10, 15)])
# vprop[v4] = MyList([i for i in range(32, 37)])
# vprop[v5] = MyList([i for i in range(73, 76)])
# parts = g.new_vertex_property("int")
# parts[v1] = 0
# parts[v2] = 0
# parts[v3] = 1
# parts[v4] = 1
# parts[v5] = 1

# condg, condparts, vcount, ecount, avp, aep = condensation_graph(gtest, parts, avprops=[vprop])
# print(condparts.a)
# print(vcount.a)
# print(ecount.a)
# for i in range(condg.num_vertices()):
    # print(avp[0][i].arr)

In [3]:
from datetime import datetime

g = load_graph("HepPh_with_louvain_and_label.graphml")

In [4]:
import time
from collections import Counter
from tabulate import tabulate

def getModularity(G, parts_idx, edge_weights):
    return modularity(G, parts_idx, weight=edge_weights)

def moveNodes(G, parts_idx, edge_weights, vertex_degrees):
    # https://arxiv.org/pdf/1810.08473.pdf

    count = 0

    # Note that when we called this function, the vertex degrees passed here,
    # are actually the sum totals of each node in a community giving us
    # sum_tot as required for finding dQ. (See report)

    # So we make a copy of the community degrees for us to modify, so as to
    # speed up dQ calculations. Every time we move a vertex to a different
    # community, we subtract and add its degree from the community it moved
    # from and the community it moved to respectively.
    community_tots = {}
    total_edges = 0
    for comm_idx,tot in zip(parts_idx.a, vertex_degrees.a):
        community_tots[comm_idx] = tot
        total_edges += tot
    total_edges = int(total_edges/2)

    # Louvain's Part 1 Main Loop
    while True:
        # Get the modularity before we try to move each vertex.
        mod_old = getModularity(G, parts_idx, edge_weights)

        # Now go over each vertex to find it's best community
        for v in G.iter_vertices():
            # if (count == 2):
            #     print("mod:", getModularity(G, parts_idx, edge_weights))
            # Keep track of communities we have to visit along with the number
            # of edges between v and each community.
            neighbour_communities = {parts_idx[v] : 0}

            # Now go through all edges of v to find neighbouring communities.
            for s, t, weight in G.iter_out_edges(v, eprops=[edge_weights]):
                if s is v and t is v:
                    continue
                # Check if s is v
                if s is v:
                    # https://stackoverflow.com/questions/42315072/python-update-a-key-in-dict-if-it-doesnt-exist
                    # Add community if not seen before
                    if parts_idx[t] not in neighbour_communities:
                        neighbour_communities[parts_idx[t]] = weight
                    # Else add the weight
                    else:
                        neighbour_communities[parts_idx[t]] += weight
                # Do the same for t
                elif t is v:
                    # https://stackoverflow.com/questions/42315072/python-update-a-key-in-dict-if-it-doesnt-exist
                    # Add community if not seen before
                    if parts_idx[s] not in neighbour_communities:
                        neighbour_communities[parts_idx[s]] = weight
                    # Else add the weight
                    else:
                        neighbour_communities[parts_idx[s]] += weight
            # The dQ if we do not change anything is 0.
            curr_max_dQ = 0

            # The current best swap is no swap as dQ would remain 0.
            curr_best_swap = parts_idx[v]

            # Prefetch k_v_own
            k_v_own = neighbour_communities[parts_idx[v]]

            # Prefetch own community tot
            own_community_tot = community_tots[parts_idx[v]]

            # Remove v from it's community for us to do the below

            # Now go through the neighbouring communities to maximize dQ.
            # https://stackoverflow.com/questions/3294889/iterating-over-dictionaries-using-for-loops
            for community, k_v_comm in neighbour_communities.items():
                # Skip if it is our own community
                if community == parts_idx[v]:
                    continue

                # Calculate dQ
                dQ = vertex_degrees[v]*(own_community_tot - vertex_degrees[v] - community_tots[community]) + 2*total_edges*(k_v_comm - k_v_own)

                # Update our curr vals if dQ bigger than our curr max dQ
                if dQ > curr_max_dQ:
                    curr_max_dQ = dQ
                    curr_best_swap = community

            # Now update v's community if we found a better one
            if curr_max_dQ > 0:
                # First update community_tots
                community_tots[curr_best_swap] += vertex_degrees[v]
                community_tots[parts_idx[v]] -= vertex_degrees[v]

                # Now change v's community
                parts_idx[v] = curr_best_swap

        count += 1
        mod_new = getModularity(G, parts_idx, edge_weights)
        print(f"After moveNodes iteration {count}:", mod_new)
        if mod_new == mod_old:
            break
    return len(set(list(parts_idx)))

def Louvain(G):
    # https://arxiv.org/pdf/1810.08473.pdf
    # Copy to prevent unintended changes
    G_copy = G.copy()
    G_copy.set_directed(False)
    print(G_copy.is_directed())

    # Create a list of MyLists where each MyList contains the list of vertices
    # a community contains.
    master_partition = G_copy.new_vp("object")
    for idx,v in enumerate(G_copy.iter_vertices()):
        master_partition[v] = MyList([G_copy.vertex_index[idx]])

    # To keep track of the number communities
    num_partitions = G_copy.num_vertices()


    # To keep track of community index for easy swaps in moveNodes()
    partition_idxs = G_copy.new_vp("int")
    for i in range(G_copy.num_vertices()):
        partition_idxs[i] = i

    curr_iteration = 0

    # To keep track of number of intra- and inter-community edges
    edge_weights = G_copy.new_ep("int")
    edge_weights.set_value(1)


    # Louvain's Phase-2 Loop
    while True:
        # To keep track of vertex degrees
        vertex_degrees = G_copy.degree_property_map("total", weight=edge_weights)

        # Get the current modularity
        old_mod = getModularity(G_copy, partition_idxs, edge_weights)

        # Move nodes between communitites to improve modularity
        num_partitions = moveNodes(G_copy, partition_idxs, edge_weights, vertex_degrees)

        curr_iteration += 1

        # Get the new modularity
        new_mod = getModularity(G_copy, partition_idxs, edge_weights)

        print(f"After iteration {curr_iteration}", new_mod)

        # If the change in modularity was less than 1e-7, then stop
        if abs(old_mod - new_mod) < 0.0000001:
            break

        # Else condense the communities into singular nodes
        G_copy, partition_idxs, vcount, edge_weights, avp, aep = condensation_graph(G_copy, partition_idxs, avprops=[master_partition], eweight=edge_weights, self_loops=True)

        # Get the new partition sets
        master_partition = avp[0]

    final_part_idxs = G.new_vp("int")
    for part_idx,part in enumerate(master_partition):
        for node in part.arr:
            final_part_idxs[node] = part_idx
    return final_part_idxs

part_idxs = Louvain(g)
g.vp.louvain_comms = part_idxs
with open("louvaincomms.txt", "w") as f:
    for i in part_idxs:
        print(i, file=f)

g.save("HepPh_with_louvain_and_label.graphml")

from google.colab import files
files.download('louvaincomms.txt')
files.download("HepPh_with_louvain_and_label.graphml")
# Use the below instead of above to test Louvain on the famous Zachary's Karate Club Dataset
# Results are validated by https://scholarworks.sjsu.edu/cgi/viewcontent.cgi?article=1528&context=etd_projects
# g = collection.data["karate"]
# part_idxs = Louvain(g)

# https://stackoverflow.com/questions/12282232/how-do-i-count-occurrence-of-unique-values-inside-a-list
parts = Counter(part_idxs).keys() # equals to list(set(part_idxs))-
freq = Counter(part_idxs).values() # counts the elements' frequency

# https://stackoverflow.com/questions/41140647/python-printing-lists-with-tabulate
table = zip(parts, freq)
print(tabulate(table, headers=["Community Index", "Number of Nodes"]))

False
After moveNodes iteration 1: 0.0031919121880345813
After moveNodes iteration 2: 0.0037817687873146293
After moveNodes iteration 3: 0.003901177099270022
After moveNodes iteration 4: 0.003901177099270022
After iteration 1 0.003901177099270022
After moveNodes iteration 1: 0.007487909005826791
After moveNodes iteration 2: 0.0077344544574275325
After moveNodes iteration 3: 0.007803971066492475
After moveNodes iteration 4: 0.007822724461266889
After moveNodes iteration 5: 0.007822724461266889
After iteration 2 0.007822724461266889
After moveNodes iteration 1: 0.010815953348735195
After moveNodes iteration 2: 0.011269423218345299
After moveNodes iteration 3: 0.011472679139133244
After moveNodes iteration 4: 0.01159057255038439
After moveNodes iteration 5: 0.011628302211683029
After moveNodes iteration 6: 0.011628302211683029
After iteration 3 0.011628302211683029
After moveNodes iteration 1: 0.015842701173026833
After moveNodes iteration 2: 0.01617447015490675
After moveNodes iteration 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  Community Index    Number of Nodes
-----------------  -----------------
                0               2651
                1               2752
                2               3476
                3               1974
                4               3518
                5               3374
                6               4885
                7               4305
                8               3133
                9                200
               10                 25
               11                 10
               12                  3
               13                  9
               14                  3
               15                  6
               16                  4
               17                  5
               18                  4
               19                  4
